In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [2]:
# load data
df=pd.read_csv('InternetAds_norm_19.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,...,att1550,att1551,att1552,att1553,att1554,att1555,att1556,att1557,att1558,outlier
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
df.shape

(3264, 1556)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/InternetAds.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,1555]
X = df[:,0:1555]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
          # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 14:52:04,712][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1555)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1555)],y_test.shape=(1078,)
[ 2018-04-26 14:52:04,728][cascade_classifier.fit_transform] group_dims=[1555]
[ 2018-04-26 14:52:04,730][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:52:04,731][cascade_classifier.fit_transform] group_ends=[1555]
[ 2018-04-26 14:52:04,732][cascade_classifier.fit_transform] X_train.shape=(2186, 1555),X_test.shape=(1078, 1555)
[ 2018-04-26 14:52:04,750][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1555), X_cur_test.shape=(1078, 1555)
[ 2018-04-26 14:52:05,553][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.63%
[ 2018-04-26 14:52:06,645][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-26 14:52:07,776][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:52:14,568][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-04-26 14:52:14,573][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-26 14:52:14,575][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-04-26 14:52:14,579][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


Test Accuracy of ExtraTrees = 97.031540 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 14:52:17,077][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-04-26 14:52:17,082][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-26 14:52:17,084][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-04-26 14:52:17,088][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 14:52:19,735][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-04-26 14:52:19,757][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-26 14:52:19,759][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:52:19,760][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-26 14:52:19,761][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-04-26 14:52:19,781][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-04-26 14:52:20,606][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.09%
[ 2018-04-26 14:52:21,620][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.26%
[ 2018-04-26 14:52:22,700][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:52:30,810][cascade_classifier.transform] X_groups_test.shape=[(1078, 1557)]
[ 2018-04-26 14:52:30,820][cascade_classifier.transform] group_dims=[1557]
[ 2018-04-26 14:52:30,821][cascade_classifier.transform] X_test.shape=(1078, 1557)
[ 2018-04-26 14:52:30,830][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1557)


Test Accuracy of ExtraTrees = 96.660482 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche() 
    print ("Time ",format(round(tt,3))," seconds")

[ 2018-04-26 14:52:33,512][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-04-26 14:52:33,527][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-26 14:52:33,528][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:52:33,529][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-26 14:52:33,530][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-04-26 14:52:33,549][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-04-26 14:52:34,327][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.63%
[ 2018-04-26 14:52:35,410][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.26%
[ 2018-04-26 14:52:36,542][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 96.938776 %', 0, 'fois')
('Time ', '10.978', ' seconds')


[ 2018-04-26 14:52:47,940][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.63%
[ 2018-04-26 14:52:49,059][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-26 14:52:50,170][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.72%
[ 2018-04-26 14:52:51,275][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.63%
[ 2018-04-26 14:52:52,491][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.80%
[ 2018-04-26 14:52:53,603][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.72%
[ 2018-04-26 14:52:54,707][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.72%
[ 2018-04-26 14:52:55,773][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.17%


('Test Accuracy of ExtraTrees = 96.753247 %', 1, 'fois')
('Time ', '11.326', ' seconds')


[ 2018-04-26 14:53:01,692][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.63%
[ 2018-04-26 14:53:02,738][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.72%
[ 2018-04-26 14:53:03,831][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.26%
[ 2018-04-26 14:53:04,982][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.17%
[ 2018-04-26 14:53:06,024][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.80%
[ 2018-04-26 14:53:07,091][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.72%
[ 2018-04-26 14:53:08,183][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-04-26 14:53:09,259][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.71%


...

('Test Accuracy of ExtraTrees = 96.846011 %', 17, 'fois')
('Time ', '11.11', ' seconds')


[ 2018-04-26 14:56:37,643][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.72%
[ 2018-04-26 14:56:38,734][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-26 14:56:39,702][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.72%
[ 2018-04-26 14:56:40,785][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.63%
[ 2018-04-26 14:56:41,890][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.80%
[ 2018-04-26 14:56:43,139][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.72%
[ 2018-04-26 14:56:44,271][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-04-26 14:56:45,337][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.17%


('Test Accuracy of ExtraTrees = 97.031540 %', 18, 'fois')
('Time ', '11.305', ' seconds')


[ 2018-04-26 14:56:51,513][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.72%
[ 2018-04-26 14:56:52,607][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.35%
[ 2018-04-26 14:56:53,699][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.72%
[ 2018-04-26 14:56:54,852][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.17%
[ 2018-04-26 14:56:55,926][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=96.80%
[ 2018-04-26 14:56:56,938][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.72%
[ 2018-04-26 14:56:57,597][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.72%
[ 2018-04-26 14:56:58,675][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.71%


('Test Accuracy of ExtraTrees = 96.938776 %', 19, 'fois')
('Time ', '10.558', ' seconds')


# ('Test Accuracy of ExtraTrees = 97.124304 %', 3, 'fois')